Imports

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import torch.utils.data as DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Create Fuly Connected Network

In [18]:
class NN(nn.Module):
  def __init__(self,input_size,num_classes):
    super(NN,self).__init__()
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50,num_classes)
  
  def forward(self,x):
    x = f.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [19]:
# Rough Check whether the model is working
model = NN(784,10)
x = torch.randn((64,784))
print(model(x).shape)

torch.Size([64, 10])


Setting up device

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
device

device(type='cuda')

Hyperparameters

In [22]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

Load Dataset

In [23]:
train_dataset = datasets.MNIST(root = "dataset/",train = True,transform = transforms.ToTensor(),download = True)

In [24]:
train_loader = DataLoader.DataLoader(dataset=train_dataset,batch_size=64,shuffle=True)

In [25]:
test_dataset = train_dataset = datasets.MNIST(root = "dataset/",train = False,transform = transforms.ToTensor(),download = True)

In [26]:
test_loader = DataLoader.DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle = True)

Initialise N/w

In [27]:
model = NN(input_size=input_size,num_classes=num_classes).to(device)

Loss and Optimizer

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

Train N/W

In [29]:
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):
    #get data to cuda if possible 
    data = data.to(device = device)
    targets = targets.to(device = device)

    data = data.reshape(data.shape[0],-1) # Flatten into a single dimension
    #forward

    scores = model(data)
    loss = criterion(scores,targets)

    #backward
    optimizer.zero_grad()
    loss.backward()


    #gradient_descent or adam-step
    optimizer.step()

# Check the accuracy for the training step
def check_accuracy(loader,model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0],-1)

      scores = model(x)
      _,predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    print(f' Got {num_correct}/{num_samples} with accuracy ={float(num_correct)/float(num_samples)*100:.2f} ')
    model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
 Got 56045/60000 with accuracy =93.41 
Checking accuracy on test data
 Got 9323/10000 with accuracy =93.23 
